In [4]:
from tqdm import tqdm
from pprint import pprint
import json
import pandas as pd
import sys
sys.path.append('/home/ajassani/iLoveTrace')
from Trace2Tree.trace_to_tree import TraceToTree
from tree_perf import TreePerfAnalyzer

In [5]:
path = '/home/ajassani/trace_data/jan6/MI325/gpt-3-xl_single-gpu_bfloat16_bs5_steps25_det0_prof1_level1/pytorch_profile_rank0.json'
with open(path, 'r') as f:
    data = json.load(f)

events = data['traceEvents']
tree = TraceToTree(events)
tree.build_tree(add_python_func=False)
perf_analyzer = TreePerfAnalyzer(tree)

Building tree with add_python_func=False
Building CPU op tree with add_python_func=False


In [6]:
gemm_events = [event for event in tree.events if event['name'] in ['aten::addmm', 'aten::mm']]
gemm_events_uids = [event['UID'] for event in gemm_events]
df_gemm_ops = perf_analyzer.build_df_perf_metrics(gemm_events_uids, bwd=False, non_data_mov=True)
perf_analyzer.summarize_df_perf_metrics(df_gemm_ops, ['mean'])

,name,param: M,param: N,param: K,param: bias,GFLOPS_first,FLOPS/Byte_first,TFLOPS/s_mean,Non-Data-Mov TFLOPS/s_mean,Non-Data-Mov Kernel Time (µs)_sum,Kernel Time (µs)_sum,name_count
1,aten::addmm,2048,10240,2048,True,85.920317,930.929747,477.313319,568.031389,14849.550,17664.548,98
5,aten::mm,2048,2048,10240,False,85.899346,930.909091,495.574488,495.574488,17066.623,17066.623,98
3,aten::addmm,8192,10240,2048,True,343.681270,1412.639696,532.749538,627.594927,13145.159,15485.034,24
9,aten::mm,2048,10240,2048,False,85.899346,930.909091,566.363674,566.363674,14938.815,14938.815,98
13,aten::mm,8192,10240,2048,False,343.597384,1412.413793,592.384987,592.384987,13957.262,13957.262,24
10,aten::mm,2048,10240,8192,False,343.597384,1412.413793,647.675987,647.675987,12736.507,12736.507,24
2,aten::addmm,2048,10240,8192,True,343.618355,1412.381097,655.723876,693.409852,11907.729,12590.310,24
7,aten::mm,2048,8192,10240,False,343.597384,1412.413793,771.075568,771.075568,10696.483,10696.483,24
12,aten::mm,8192,2048,10240,False,343.597384,1412.413793,807.936044,807.936044,10221.309,10221.309,24
11,aten::mm,2048,55200,2048,False,463.051162,1005.350028,601.501171,601.501171,3098.930,3098.930,4
